In [ ]:
import nbimporter
import random
from imports import *
from functions_uc6 import *
pd.set_option('max_colwidth', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
# load data
data_ume_ = pd.read_csv("./YOUR_PATH/YOUR_FILE.csv", delimiter=';')

In [ ]:
# data preprocessing
data_ume_scaled, scaler_ume = process_use_case_3_data_with_scaling(data_ume_, start_date_ume, end_date_ume)

In [ ]:
# Recreate rolling windows 
X, y, prediction_start_dates,X_df,y_df,full_data = create_rolling_windows_with_weekday_prediction(
    data_ume_scaled, target_col="shift_count", lookback=42, horizon=28, prediction_days=['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday','Saturday','Sunday','Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday','Saturday','Sunday','Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday','Saturday','Sunday','Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday','Saturday','Sunday']
)

In [ ]:
def timellm_predict(full_data,scaler):
    full_data_test = full_data.groupby("id").tail(21)
    full_data_train = full_data.drop(full_data_test.index)
    full_data_train_ = full_data_train.rename(columns={'date': 'ds', 'id': 'unique_id','value':'y'})
    full_data_test_ = full_data_test.rename(columns={'date': 'ds', 'id': 'unique_id','value':'y'})
    full_data= full_data.rename(columns={'date': 'ds', 'id': 'unique_id','value':'y'})
    full_data_train_ = full_data_train_.drop(columns=['type'])
    full_data_test_ = full_data_test_.drop(columns=['type'])
    random.seed(42)
    random_encounter_ids = random.sample(list(full_data_train_['unique_id']), 5000) # change for no fine-tuning
    # Filtering out the selected encounter IDs
    full_data_train_limit = full_data_train_[full_data_train_['unique_id'].isin(random_encounter_ids)]
    full_data_test_limit = full_data_test_[full_data_test_['unique_id'].isin(random_encounter_ids)]
    prompt_prefix = "The dataset containts time series of CT-scan procedures. Values are min-max scaled. Predicted values can't be negative. The prediction should be done for the following 28 days with 1D sampling frequency. Predictions should be float. Dont make predicted values too similar, addd amplitudes and spikes!"
    timellm = TimeLLM(
        h=28,
        input_size=42,
        llm='meta-llama/Llama-2-7b-hf',
        prompt_prefix=prompt_prefix,
        batch_size=24,
        d_llm=4096,
        valid_batch_size=24,
        windows_batch_size=24,
        max_steps=100,
        learning_rate=1e-4
    )

    nf = NeuralForecast(
        models=[timellm],
        freq='D'
        )
    nf.fit(df=full_data_train_limit)
    forecasts = nf.predict(full_data_train_)
    
    def rescale_to_original(scaled_df, scaler, columns_to_rescale):
        scaled_df[columns_to_rescale] = scaler.inverse_transform(scaled_df[columns_to_rescale])
        return scaled_df
    
    columns_to_rescale_actual = ["y"]
    columns_to_rescale_predict = ["TimeLLM"]
    actuals = rescale_to_original(full_data_test_, scaler,columns_to_rescale_actual)
    predictions = rescale_to_original(forecasts, scaler,columns_to_rescale_predict)
    return predictions,actuals,nf